# **Lab 2: QR decomposition**
### **Họ và tên:** Đỗ Tiến Đạt
### **MSSV:** 23120119
### **Lớp:** 23CTT2

# **Một số lưu ý trước khi thực thi mã nguồn**
- Phải thực thi phần cell code chứa `class Matrix` và `class Vector` trước tiên để giải thuật có thể thực hiện

In [ ]:
import copy

# Vector
class Vector:
  def __init__(self, data : list[float]):
    self.data = data
    self.length = len(data)

  def norm(self) -> float:
    return sum(self.data[i] ** 2 for i in range(self.length)) ** 0.5

  def dot(self, other : "Vector") -> float:
    if self.length != other.length:
      raise ValueError("Cannot take a dot product")
    return sum(self.data[i] * other.data[i] for i in range(self.length))

  def multiply_scalar(self, s : float) -> "Vector":
    return Vector([self.data[i] * s for i in range(self.length)])

  def subtract_v_by_v(self, v : "Vector") -> "Vector":
    _v = copy.deepcopy(self)
    if _v.length != v.length:
      return
    return Vector([_v.data[i] - v.data[i] for i in range(_v.length)])

  def display(self):
    for i in range(self.length):
      print(f'{self.data[i]} ')
    print('\n')

# Matrix
class Matrix:
  def __init__(self, data : list[list[float]]):
    self.row = len(data)
    self.col = len(data[0])
    self.data = data

  def add(self, other : "Matrix") -> "Matrix":
    return Matrix([[self.data[i][j] + other.data[i][j] for j in range(self.col)]
                                                       for i in range(self.row)])

  def subtract(self, other : "Matrix") -> "Matrix":
    return Matrix([[self.data[i][j] - other.data[i][j] for j in range(self.col)]
                                            for i in range(self.row)])

  def multiply(self, other: "Matrix") -> "Matrix":
    if self.col != other.row:
        raise ValueError("Cannot multiply: Column of first matrix must equal row of second matrix.")

    return Matrix([[
        sum(self.data[i][k] * other.data[k][j] for k in range(self.col))
        for j in range(other.col)
    ] for i in range(self.row)])

  def tranpose(self) -> "Matrix":
    return Matrix([[self.data[i][j] for j in range(self.col)]
                                    for i in range(self.row)])

  def replace_vc_by_vc(self, vc : Vector, col_idx : int):
    if col_idx < 0 or col_idx >= self.col or self.row != vc.length:
      raise ValueError("Columns index out of bounds or vector length mismatch")
    for i in range(self.row):
      self.data[i][col_idx] = vc.data[i]

  def extract_col(self, col_idx : int) -> Vector:
    return Vector([self.data[i][col_idx] for i in range(self.row)])

  def __str__(self) -> str:
        return "\n".join(["\t".join(f"{x:.3f}" for x in row) for row in self.data])

# Giải thuật Gram-Schmidt

### - Input:
Hệ vector độc lập tuyến tính $u_1, u_2, \dots, u_n$
### - Output:
Hệ vector trực giao $v_1, v_2, \dots, v_n$ sao cho $span(u_1, u_2, \dots, u_h) = span(v_1, v_2, \dots, v_h)$, $ \forall h = 1,...,n$

### - Các bước thực hiện
Lần lượt thực hiện $n$ bước sau
- **Bước 1:** Đặt $v_1 = \frac{u_1}{\|u_1\|}$
- **Bước 2:** Đặt $v_2 = u_2 - \langle u_2, v_1
⟩ v_1$
- **Bước 3:** Đặt $v_3 = u_3 - \langle u_3, v_1 \rangle v_1 - \langle u_3, v_2 \rangle v_2$ <br>
$\dots$
- **Bước n:** Đặt $v_n = u_n - \langle u_n, v_1\rangle v_1 - \langle u_n, v_2\rangle v_2 - \dots - \langle u_n, v_{n-1}\rangle v_{n-1}$

Khi đấy ta thu được hệ các vector $v_1, v_2, \dots, v_n$ trực giao thỏa mãn $span(u_1, u_2, \dots, u_h) = span(v_1, v_2, \dots, v_h)$, $ \forall h = 1,...,n$.

# Phân rã QR

## Ý tưởng thực hiện

1. **Trực giao hóa các vector cột**:
   - Ban đầu, tất cả các vector cột $u_1, u_2, \dots, u_n$ trong ma trận $A$ có thể không vuông góc với nhau. Thuật toán Gram-Schmidt sẽ loại bỏ phần song song của mỗi vector so với các vector trước đó, để các vector này trở thành trực giao.

2. **Tạo ra các vector đơn vị**:
   - Sau khi loại bỏ phần song song, ta chuẩn hóa các vector trực giao $v_i$ để có độ dài bằng 1. Các vector chuẩn hóa này tạo thành các cột của ma trận $Q$.

3. **Tính toán ma trận $R$**:
   - Các hệ số chiếu giữa các vector $u_i$ và các vector trực giao $v_k$ được lưu vào ma trận $R$. Ma trận $R$ có dạng tam giác trên với các giá trị chiếu tại các vị trí trên đường chéo chính và các giá trị chiếu giữa các vector trên các vị trí ngoài đường chéo.

4. **Kết quả phân rã**:
   - Sau khi thực hiện đầy đủ các bước trên, ta có được ma trận phân rã $A = QR$ với:
     - $Q$ là ma trận với các vector đơn vị trực giao.
     - $R$ là ma trận tam giác trên, chứa các hệ số chiếu.
---
## Các bước thực hiện
### - Input:
Cho ma trận $A = [u_1, u_2, \dots, u_n]$ với $u_i$ là các vector cột và hệ $u_1, u_2, \dots, u_n$ độc lập tuyến tính.

### - Output:
Hai ma trận $Q$ và $R$ sao cho $A = QR$

### - Các bước thực hiện

- **Bước 1:** Khởi tạo hai ma trận $Q$ và $R$:
  - Ma trận $Q$ có các cột là các vector đơn vị, trực giao với nhau.
  - Ma trận $R$ sẽ chứa các hệ số chiếu từ các vector cột của ma trận $A$ lên các vector trực giao trong $Q$.

- **Bước 2:** Với $i = 1 $ đến $n$:
  - Đặt $v_i = u_i$
  - Với $k = 1$ đến $i - 1$:
    - Tính giá trị $R[k, i] := \langle u_i, v_k \rangle$, đây là hệ số chiếu của $u_i$ trên $v_k$.
    - Cập nhật $v_i := v_i - \langle u_i, v_k \rangle v_k$
  - Tính $R[i, i] := \|v_i\|$
  - Chuẩn hóa vector  $v_i := \frac{v_i}{\|v_i\|}$, sau đó lưu kết quả vào ma trận $Q[:, i]$.




In [ ]:
def qr_decomposition(A: Matrix) -> Matrix:
  m, n = A.row, A.col # number of rows, cols

  # initialize Q and R
  Q = Matrix([[0] * n for _ in range(m)])
  R = Matrix([[0] * n for _ in range(n)])

  for i in range(n):
    c = A.extract_col(i) # take the i-th column vector of A
    f = c

    for k in range(i):
      e_k = Q.extract_col(k)
      R.data[k][i] = c.dot(e_k) # calculate R[k, i]
      t = R.data[k][i]
      f = f.subtract_v_by_v(e_k.multiply_scalar(t))

    R.data[i][i] = f.norm() # calculate R[i, i]
    e = f.multiply_scalar(1 / f.norm()) # normalize the vector
    Q.replace_vc_by_vc(e, i) # subtitute the i-th column of Q

  return Q, R

### Một số các ví dụ

In [ ]:
# Example matrix A
A = Matrix(
    data = [[1, 0, 0],
            [1, 1, 0],
            [1, 1, 1]]
)
Q, R = qr_decomposition(A)
print("Q (Orthonormal Matrix):")
print(Q)
print("\nR (Upper Triangular Matrix):")
print(R)

Q (Orthonormal Matrix):
0.577	-0.816	-0.000
0.577	0.408	-0.707
0.577	0.408	0.707

R (Upper Triangular Matrix):
1.732	1.155	0.577
0.000	0.816	0.408
0.000	0.000	0.707


In [ ]:
# Example matrix B
B = Matrix(
    data = [[1, 1, 1],
            [2, 2, 0],
            [3, 0, 0],
            [0, 0, 1]]
)
Q, R = qr_decomposition(B)
print("Q (Orthonormal Matrix):")
print(Q)
print("\nR (Upper Triangular Matrix):")
print(R)

Q (Orthonormal Matrix):
0.267	0.359	0.596
0.535	0.717	-0.298
0.802	-0.598	0.000
0.000	0.000	0.745

R (Upper Triangular Matrix):
3.742	1.336	0.267
0.000	1.793	0.359
0.000	0.000	1.342


In [ ]:
# Example matrix B
C = Matrix(
    data = [[12, -51, 4],
            [6, 167, -68],
            [-4, 24, -41]]
)
Q, R = qr_decomposition(C)
print("Q (Orthonormal Matrix):")
print(Q)
print("\nR (Upper Triangular Matrix):")
print(R)

Q (Orthonormal Matrix):
0.857	-0.394	-0.331
0.429	0.903	0.034
-0.286	0.171	-0.943

R (Upper Triangular Matrix):
14.000	21.000	-14.000
0.000	175.000	-70.000
0.000	0.000	35.000


## Mô tả các hàm thực hiện
- `class Vector` và `class Matrix` cung cấp các thuộc tính và phương thức cho việc thực hiện các phép toán trên ma trận.
- `qr_decomposition` thực hiện phân rã ma trận $A$ ra thành 2 ma trận $Q$, $R$ (các bước thực hiện đã được nêu rõ ở bên trên)

# Mở rộng

## Dưới đây là một cách hiện thực giải thuật Gram-Schmidt sử dụng thư viện `numpy`



In [ ]:
import numpy as np

def gram_schmidt(A):
  m, n = A.shape # number of rows, cols


  Q = np.zeros((m, n)) # initialize Q and R
  R = np.zeros((n, n))

  for i in range(n):
    a_i = A[:, i] # Take the i-th column of A

    for k in range(i):
      R[k, i] = np.dot(Q[:, k], a_i)  # calculate R[k, i]
      a_i = a_i - R[k, i] * Q[:, k]

    R[i, i] = np.linalg.norm(a_i)  # calculate R[i, i]
    Q[:, i] = a_i / R[i, i]  # normalize the vector

  return Q, R

In [ ]:
# Example matrix A
A = np.array([[1, 0, 0],
              [1, 1, 0],
              [1, 1, 1]], dtype = float)

Q, R = gram_schmidt(A)
print("Q (Orthonormal Matrix):")
print(Q)
print("\nR (Upper Triangular Matrix):")
print(R)

Q (Orthonormal Matrix):
[[ 0.57735027 -0.81649658  0.        ]
 [ 0.57735027  0.40824829 -0.70710678]
 [ 0.57735027  0.40824829  0.70710678]]

R (Upper Triangular Matrix):
[[1.73205081 1.15470054 0.57735027]
 [0.         0.81649658 0.40824829]
 [0.         0.         0.70710678]]


In [ ]:
# Example matrix B
B = np.array([[1, 1, 1],
              [2, 2, 0],
              [3, 0, 0],
              [0, 0, 1]], dtype = float)

Q, R = gram_schmidt(B)
print("Q (Orthonormal Matrix):")
print(Q)
print("\nR (Upper Triangular Matrix):")
print(R)

Q (Orthonormal Matrix):
[[ 2.67261242e-01  3.58568583e-01  5.96284794e-01]
 [ 5.34522484e-01  7.17137166e-01 -2.98142397e-01]
 [ 8.01783726e-01 -5.97614305e-01  2.06877846e-17]
 [ 0.00000000e+00  0.00000000e+00  7.45355992e-01]]

R (Upper Triangular Matrix):
[[3.74165739 1.33630621 0.26726124]
 [0.         1.79284291 0.35856858]
 [0.         0.         1.34164079]]


In [ ]:
C = np.array([[12, -51, 4],
              [6, 167, -68],
              [-4, 24, -41]], dtype = float)

Q, R = gram_schmidt(C)
print("Q (Orthonormal Matrix):")
print(Q)
print("\nR (Upper Triangular Matrix):")
print(R)

Q (Orthonormal Matrix):
[[ 0.85714286 -0.39428571 -0.33142857]
 [ 0.42857143  0.90285714  0.03428571]
 [-0.28571429  0.17142857 -0.94285714]]

R (Upper Triangular Matrix):
[[ 14.  21. -14.]
 [  0. 175. -70.]
 [  0.   0.  35.]]


## Thư viện `numpy` cũng có phương thức cho việc phân rã ma trận A thành 2 ma trận Q, R. Tuy nhiên, giải thuật mà thư viện `numpy` sử dụng là *Householder transformations* hoặc *Givens rotations*, có thể phân rã thành 2 ma trận Q, R kể cả khi các cột của ma trận A không độc lập tuyến tính.

In [ ]:
import numpy as np

# matrix A
A = np.array([[1, 0, 0],
              [1, 1, 0],
              [1, 1, 1]], dtype=float)

# Perform QR decomposition using numpy
Q, R = np.linalg.qr(A)

print("\nQ (Orthonormal Matrix):")
print(Q)

print("\nR (Upper Triangular Matrix):")
print(R)

# Verify that A = Q * R
print("\nVerification (A should equal Q * R):")
print(np.allclose(A, np.dot(Q, R)))  # Should output True


Q (Orthonormal Matrix):
[[-5.77350269e-01  8.16496581e-01 -6.99362418e-17]
 [-5.77350269e-01 -4.08248290e-01 -7.07106781e-01]
 [-5.77350269e-01 -4.08248290e-01  7.07106781e-01]]

R (Upper Triangular Matrix):
[[-1.73205081 -1.15470054 -0.57735027]
 [ 0.         -0.81649658 -0.40824829]
 [ 0.          0.          0.70710678]]

Verification (A should equal Q * R):
True


In [31]:
import numpy as np

# matrix B
B = np.array([[1, 1, 0],
              [1, 1, 0],
              [1, 1, 1]], dtype=float)

# Perform QR decomposition using numpy
Q, R = np.linalg.qr(B)

print("\nQ (Orthonormal Matrix):")
print(Q)

print("\nR (Upper Triangular Matrix):")
print(R)

# Verify that A = Q * R
print("\nVerification (A should equal Q * R):")
print(np.allclose(A, np.dot(Q, R)))  # Should output True


Q (Orthonormal Matrix):
[[-0.57735027 -0.57735027 -0.57735027]
 [-0.57735027  0.78867513 -0.21132487]
 [-0.57735027 -0.21132487  0.78867513]]

R (Upper Triangular Matrix):
[[-1.73205081 -1.73205081 -0.57735027]
 [ 0.          0.         -0.21132487]
 [ 0.          0.          0.78867513]]

Verification (A should equal Q * R):
True


# Ứng dụng

## I. Giải các hệ phương trình tuyến tính
Phân rã QR là một cách để giải phương trình $Ax = b$, khi việc xác định ma trận nghịch đảo của $A$ khó khăn. Vì $Q$ là ma trận trực giao nên $Q^TQ = I$, do đó $Ax = b \Leftrightarrow Rx = Q^Tb $ tính toán đơn giản hơn do $R$ là ma trận tam giác trên.

## II.Giải bài toán hồi quy tuyến tính
### 1. **Bài toán hồi quy tuyến tính**:
- **Mục tiêu**: Dự đoán giá trị $y$ (biến phụ thuộc) dựa trên một tập hợp các đặc trưng $X$ (biến độc lập).

$$
y = Xw + b
$$

### 2. **Giải quyết bằng phân rã QR**:
- Phương pháp để tìm $w$ là tổi thiểu hóa bình phương sai số. Ta đưa về giải bài toán sau:
$$
w^* = \underset{w}{\text{min}} \| Xw - y \|^2
$$
- Khi đấy ta chỉ cần giải phương trình $X^TXw = X^Ty$ để tìm $w^*$
- Thay vì sử dụng phương pháp thông thường để tính nghịch đảo ma trận $X^TX$, ta sử dụng phân rã QR để tách ma trận $X$ thành hai ma trận trực giao $Q$ và tam giác trên $R$, giúp giải phương trình một cách ổn định và hiệu quả.

$$
X = QR
$$

Sau đó, bài toán hồi quy tuyến tính trở thành giải hệ phương trình:

$$
Rw = Q^TX^Ty
$$

Việc giải phương trình này sẽ dễ dàng hơn vì $R$ là ma trận tam giác trên, có thể giải bằng phương pháp thay thế ngược.


## III.Xử lý ảnh
### 1. **Ứng dụng trong phân tích hình ảnh**:
Phân rã QR có thể được sử dụng trong các kỹ thuật như **giảm chiều** (dimensionality reduction) trong xử lý ảnh, đặc biệt khi làm việc với các tập dữ liệu lớn hoặc các ma trận đặc trưng của ảnh có kích thước lớn.

Giả sử ta có một tập hợp các hình ảnh, mỗi hình ảnh có thể được biểu diễn dưới dạng ma trận (ảnh xám hoặc ma trận các giá trị pixel). Mỗi cột của ma trận có thể là một vector đặc trưng của một ảnh. Phân rã QR có thể giúp tách ma trận đặc trưng thành hai ma trận trực giao ($Q$) và tam giác trên ($R$), từ đó giảm thiểu việc tính toán nghịch đảo ma trận trong các phương pháp tiếp theo.

### 2. **Phân tích thành phần chính (PCA) với QR**:
Phân rã QR có thể được sử dụng như một bước trong việc thực hiện **Phân tích thành phần chính (PCA)**, một kỹ thuật giảm chiều quan trọng trong xử lý ảnh và học máy.

PCA giúp giảm số lượng đặc trưng trong các tập dữ liệu ảnh mà không làm mất nhiều thông tin quan trọng. Quy trình này có thể bao gồm:
- **Bước 1**: Chuẩn hóa dữ liệu (trừ đi trung bình của mỗi đặc trưng).
- **Bước 2**: Tính ma trận hiệp phương sai của các đặc trưng.
- **Bước 3**: Sử dụng phân rã QR để phân tách ma trận hiệp phương sai thành các ma trận trực giao ($Q$) và tam giác trên ($R$).
- **Bước 4**: Sử dụng ma trận $Q$ để trích xuất các thành phần chính.

Điều này giúp cho việc giảm chiều dữ liệu, giữ lại thông tin quan trọng trong ảnh và loại bỏ các yếu tố không cần thiết.

### 3. **Ứng dụng trong xử lý ảnh**:
   - **Giảm chiều và nén ảnh**: Phân rã QR có thể được sử dụng để giảm chiều các dữ liệu ảnh, giúp giảm kích thước của các tập dữ liệu lớn và tiết kiệm bộ nhớ trong các hệ thống xử lý ảnh.
   - **Phát hiện và phân loại ảnh**: QR có thể hỗ trợ trong các hệ thống phát hiện vật thể và phân loại ảnh bằng cách làm giảm chiều của các dữ liệu hình ảnh, giúp các mô hình học máy làm việc hiệu quả hơn.

### 4. **Ví dụ ứng dụng thực tế**:
   - **Khôi phục ảnh**: Trong khôi phục ảnh (image reconstruction), phân rã QR có thể giúp xác định các hệ số tối ưu từ các ảnh bị nhiễu hoặc mất thông tin.
   - **Nhận dạng khuôn mặt**: QR được sử dụng để giảm chiều các đặc trưng khuôn mặt trong các hệ thống nhận dạng, giúp tăng tốc quá trình phân tích và nhận diện.
   - **Mô hình hóa ảnh**: Phân rã QR hỗ trợ trong việc giảm số lượng đặc trưng của ảnh khi cần phân tích hoặc trích xuất các đặc trưng quan trọng từ ảnh.

# Tài liệu tham khảo
[Reference: QR Factorization - LibreTexts] https://math.libretexts.org/Bookshelves/Linear_Algebra/Linear_Algebra_with_Applications_(Nicholson)/08%3A_Orthogonality/8.04%3A_QR-Factorization

